# aii) Gender Classification

In [37]:
import pandas as pd

df = pd.read_csv('../Datasets/ADS/final_training_data.csv', index_col=0)


In [38]:
df.head()

,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope,...,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp,participant_id,Depression,Gender
0,33.883310,0.188470,31.197649,31.198807,31.237043,0.039394,39.990345,28.040092,100.798780,0.000000,...,1.960784,2.061856,0.240000,0.010000,0.225,0.005000,-53.529380,448,1,1
1,23.442284,0.015231,23.083265,23.519197,23.799660,0.716394,0.000000,0.000000,-0.109734,0.000000,...,3.703704,2.040816,0.080000,0.000000,0.380,0.000000,-47.326970,448,1,1
2,26.658195,0.095594,24.275490,27.404346,29.017082,4.741592,61.829530,67.673560,20.461290,11.705440,...,3.131991,2.036199,0.344444,0.235472,0.115,0.087321,-41.121784,448,1,1
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.333334,0.000000,0.000000,0.000000,0.240,0.000000,-56.265000,448,1,1
4,34.614662,0.005980,34.433628,34.559757,34.764160,0.330532,8.279264,5.828207,8.208370,6.596444,...,5.084746,1.886793,0.420000,0.000000,0.040,0.020000,-33.531155,448,1,1


In [110]:
# print null count
print(df.isnull().sum())

# Print row with null values
print(df[df.isnull().any(axis=1)])

F0semitoneFrom27.5Hz_sma3nz_amean             0
F0semitoneFrom27.5Hz_sma3nz_stddevNorm        0
F0semitoneFrom27.5Hz_sma3nz_percentile20.0    0
F0semitoneFrom27.5Hz_sma3nz_percentile50.0    0
F0semitoneFrom27.5Hz_sma3nz_percentile80.0    0
                                             ..
StddevUnvoicedSegmentLength                   0
equivalentSoundLevel_dBp                      0
participant_id                                0
Depression                                    0
Gender                                        0
Length: 91, dtype: int64
Empty DataFrame
Columns: [F0semitoneFrom27.5Hz_sma3nz_amean, F0semitoneFrom27.5Hz_sma3nz_stddevNorm, F0semitoneFrom27.5Hz_sma3nz_percentile20.0, F0semitoneFrom27.5Hz_sma3nz_percentile50.0, F0semitoneFrom27.5Hz_sma3nz_percentile80.0, F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2, F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope, F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope, F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope, F0semitoneFrom27.5Hz_sma3nz_stddev

# Preprocessing

In [40]:
# Drop null rows
df = df.dropna()

# Training

In [42]:
from sklearn.model_selection import train_test_split

# Drop Depression, Gender columns
X = df.drop(['Depression', 'Gender', 'participant_id'], axis=1)
y = df['Gender']

# normalize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

#### Feed-Forward Neural Network

In [111]:
from keras import models, layers, optimizers, regularizers

def build_and_train_model(layer_sizes, activation_fn, regularization_rate, X_train, y_train, X_val, y_val):
    model = models.Sequential()
    for i, size in enumerate(layer_sizes):
        if i == 0:
            model.add(layers.Dense(size, activation=activation_fn, input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l2(regularization_rate)))
        else:
            model.add(layers.Dense(size, activation=activation_fn, kernel_regularizer=regularizers.l2(regularization_rate)))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=0)

    if X_val is not None and y_val is not None:
        _, accuracy = model.evaluate(X_val, y_val, verbose=0)
        print('Validation Accuracy:', accuracy)
        
    return model

model1 = build_and_train_model([256, 64, 32], 'relu', 0.001, X_train, y_train, X_val, y_val)
model2 = build_and_train_model([256, 64, 32], 'sigmoid', 0.001, X_train, y_train, X_val, y_val)
model3 = build_and_train_model([256, 128, 64, 32], 'relu', 0.001, X_train, y_train, X_val, y_val)
model4 = build_and_train_model([512, 256, 128, 32], 'relu', 0.001, X_train, y_train, X_val, y_val)


Validation Accuracy: 0.9599999785423279
Validation Accuracy: 0.9420183300971985
Validation Accuracy: 0.9647706151008606
Validation Accuracy: 0.9566972255706787


#### Random Forest

In [50]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_features': ['sqrt', 'log2'],  # Number of features to consider at every split
    'max_depth': [None, 20, 40],  # Maximum number of levels in tree
    'min_samples_split': [5, 10]  # Minimum number of samples required to split a node
}

# Initialize the classifier
rf = RandomForestClassifier(random_state=42)

# Setup the grid search with 5-fold cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

# Fit grid_search to the data
grid_search.fit(X_train, y_train)


# Best parameters
best_params = grid_search.best_params_
print("Best parameters:", best_params)

# Best model from grid search
best_rf = grid_search.best_estimator_

# Predict on the validation set
y_pred = best_rf.predict(X_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)


Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 300}
Validation Accuracy: 0.9467889908256881


output (21m 22.7s):
```
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 300}
Validation Accuracy: 0.9467889908256881
```

# Choosing a ML Algorithm/Model
We will choose to use the FFNN Model 3 architecture as it achieved the best sample level validation accuracy.

In [113]:
final_model = build_and_train_model([256, 128, 64, 32], 'relu', 0.001, X, y, None, None)

# Accuracies at the Participant Level

In [114]:
# Split data frame into multiple dataframes by participant_id and store a list of dataframes
def split_df_by_id(df):
    return [df[df['participant_id'] == id] for id in df['participant_id'].unique()]


In [115]:
def classifyParticipantGender(participant_df, model):
    # Drop Depression, Gender, participant_id
    X = participant_df.drop(['Depression','Gender', 'participant_id'], axis=1)
    y = participant_df['Gender'].values[0]

    # Normalize the data according to the scaler used for training
    X = scaler.transform(X)
    
    # Predict
    y_pred = model.predict(X, verbose=0)

    # Majority vote for the participant
    y_pred = y_pred.mean() > 0.5

    return y_pred, y

In [116]:
def getAccuraciesForParticipants(df_list, model):
    # female = 0, male = 1
    correct_female = 0
    correct_male = 0
    total_female = 0
    total_male = 0
    for df in df_list:
        pred, actual = classifyParticipantGender(df, model)
        if pred == actual:
            if actual == 0:
                correct_female += 1
            else:
                correct_male += 1
        if actual == 0:
            total_female += 1
        else:
            total_male += 1

    accuracy = (correct_female + correct_male) / (total_female + total_male)
    balanced_accuracy = 0.5 * correct_female / total_female + 0.5 * correct_male / total_male
                
    return accuracy, balanced_accuracy

In [117]:
acc, bal_acc = getAccuraciesForParticipants(split_df_by_id(df), final_model)
print("Training Accuracy:", acc, "Balanced Accuracy:", bal_acc)

Training Accuracy: 1.0 Balanced Accuracy: 1.0


In [118]:
test_df = pd.read_csv('../Datasets/ADS/final_testing_data.csv', index_col=0)

In [119]:
acc, bal_acc = getAccuraciesForParticipants(split_df_by_id(test_df), final_model)
print("Training Accuracy:", acc, "Balanced Accuracy:", bal_acc)

Training Accuracy: 1.0 Balanced Accuracy: 1.0


# Results

100% accuracy and 100% balanced accuracy on both the training set and the test set.